# Настройка
Все зависимости перечислены в ячейке ниже. Кроме того, есть ещё дополнительные данные (opencorpora, например). Они тоже скачиваются в первых ячейках.

In [1]:
%%writefile requirements.txt
nltk>=3.4.5
razdel>=0.4.0
rusenttokenize>=0.0.5
b-labs-models>=2017.8.22
lxml>=4.2.1
spacy>=2.1.4
pymystem3>=0.2.0
rnnmorph>=0.4.0

Overwriting requirements.txt


In [2]:
import sys
# !pip install --user --upgrade --force-reinstall -r requirements.txt

In [ ]:
# Restart kernel

import os
os._exit(0)

In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import sys
!python3 -m spacy download en_core_web_sm

2023-03-07 12:40:36.480221: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 13.9 MB 1.3 MB/s            
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


### Дополнительные данные

Opencorpora: 31 Мб по сети, 530 Мб в распакованном виде

In [1]:
!wget http://opencorpora.org/files/export/annot/annot.opcorpora.xml.bz2

--2023-03-08 10:11:22--  http://opencorpora.org/files/export/annot/annot.opcorpora.xml.bz2
Resolving opencorpora.org (opencorpora.org)... 164.92.197.18
Connecting to opencorpora.org (opencorpora.org)|164.92.197.18|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32754997 (31M) [application/x-bzip2]
Saving to: ‘annot.opcorpora.xml.bz2’

annot.opcorpora.xml 100%[===================>]  31.24M  41.3MB/s    in 0.8s    

2023-03-08 10:11:23 (41.3 MB/s) - ‘annot.opcorpora.xml.bz2’ saved [32754997/32754997]



In [2]:
!bzip2 -d annot.opcorpora.xml.bz2

### Тестовые примеры

In [1]:
example1 = "this's a sent tokenize test. this is sent two. is this sent three? sent 4 is cool! Now it’s your turn."
example2 = """
    An ambitious campus expansion plan was proposed by Fr. Vernon F. Gallagher in 1952.
    Assumption Hall, the first student dormitory, was opened in 1954,
    and Rockwell Hall was dedicated in November 1958, housing the schools of business and law.
    It was during the tenure of F. Henry J. McAnulty that Fr. Gallagher's ambitious plans were put to action.
"""
example3 = """
    А что насчёт русского языка? Хорошо ли сегментируются имена?
    Ай да А.С. Пушкин! Ай да сукин сын!
    «Как же так?! Захар...» — воскликнут Пронин.
    - "Так в чем же дело?" - "Не ра-ду-ют".
    И т. д. и т. п. В общем, вся газета.
    Православие... более всего подходит на роль такой идеи...
    Нефть за $27/барр. не снится.
"""
example4 = """
    Кружка-термос на 0.5л (50/64 см³, 516;...) стоит $3.88
"""
example5 = """
    Good muffins cost $3.88 in New York.  Please buy me two of them. Thanks.
"""

# Сегментация предложений
Первая задача - разбиение текста на предложения

### Экперименты

##### NLTK - Natural Language Toolkit
Популярная платформа для анализа текстов. Особенно хорошо работает для английского. В основном не содержит ничего из машинного обучения, только старые добрые правила.

In [2]:
from nltk.tokenize import sent_tokenize
sent_tokenize(example1)

["this's a sent tokenize test.",
 'this is sent two.',
 'is this sent three?',
 'sent 4 is cool!',
 'Now it’s your turn.']

А вот тут что-то пошло не так

In [3]:
sent_tokenize(example2)

['\n    An ambitious campus expansion plan was proposed by Fr.',
 'Vernon F. Gallagher in 1952.',
 'Assumption Hall, the first student dormitory, was opened in 1954,\n    and Rockwell Hall was dedicated in November 1958, housing the schools of business and law.',
 'It was during the tenure of F. Henry J. McAnulty that Fr.',
 "Gallagher's ambitious plans were put to action."]

А что насчёт русского языка?

In [4]:
sent_tokenize(example3)

['\n    А что насчёт русского языка?',
 'Хорошо ли сегментируются имена?',
 'Ай да А.С.',
 'Пушкин!',
 'Ай да сукин сын!',
 '«Как же так?!',
 'Захар...» — воскликнут Пронин.',
 '- "Так в чем же дело?"',
 '- "Не ра-ду-ют".',
 'И т. д. и т. п. В общем, вся газета.',
 'Православие... более всего подходит на роль такой идеи...\n    Нефть за $27/барр.',
 'не снится.']

https://github.com/Mottl/ru_punkt

Data for sentence tokenization was taken from 3 sources:

  * Articles from Russian Wikipedia (about 1 million sentences)
  * Common Russian abbreviations from Russian orthographic dictionary, edited by V. V. Lopatin;
  * Generated names initials.

In [5]:
sent_tokenize(example3, language="russian")

['\n    А что насчёт русского языка?',
 'Хорошо ли сегментируются имена?',
 'Ай да А.С. Пушкин!',
 'Ай да сукин сын!',
 '«Как же так?!',
 'Захар...» — воскликнут Пронин.',
 '- "Так в чем же дело?"',
 '- "Не ра-ду-ют".',
 'И т. д. и т. п. В общем, вся газета.',
 'Православие... более всего подходит на роль такой идеи...\n    Нефть за $27/барр.',
 'не снится.']

https://github.com/natasha/razdel

razdel старается разбивать текст на предложения и токены так, как это сделано в 4 датасетах: SynTagRus, OpenCorpora, ГИКРЯ и РНК из репозитория morphoRuEval-2017.

В основном это новостные тексты и литература. Правила razdel заточены под них.

На текстах другой тематики (социальные сети, научные статьи) библиотека может работать хуже.

In [6]:
! pip install razdel

In [7]:
from razdel import sentenize
list(sentenize(example3))

[Substring(5, 33, 'А что насчёт русского языка?'),
 Substring(34, 65, 'Хорошо ли сегментируются имена?'),
 Substring(70, 88, 'Ай да А.С. Пушкин!'),
 Substring(89, 105, 'Ай да сукин сын!'),
 Substring(110, 123, '«Как же так?!'),
 Substring(124, 154, 'Захар...» — воскликнут Пронин.'),
 Substring(159, 181, '- "Так в чем же дело?"'),
 Substring(182, 198, '- "Не ра-ду-ют".'),
 Substring(203, 218, 'И т. д. и т. п.'),
 Substring(219, 239, 'В общем, вся газета.'),
 Substring(244,
           301,
           'Православие... более всего подходит на роль такой идеи...'),
 Substring(306, 335, 'Нефть за $27/барр. не снится.')]

https://github.com/deepmipt/ru_sentence_tokenizer
    
A simple and fast rule-based sentence segmentation. Tested on OpenCorpora and SynTagRus datasets.

In [8]:
! pip install rusenttokenize

In [9]:
from rusenttokenize import ru_sent_tokenize
ru_sent_tokenize(example3)

['А что насчёт русского языка?',
 'Хорошо ли сегментируются имена?',
 'Ай да А.С. Пушкин!',
 'Ай да сукин сын!',
 '«Как же так?!',
 'Захар...» — воскликнут Пронин.',
 '- "Так в чем же дело?"',
 '- "Не ра-ду-ют".',
 'И т. д. и т. п.',
 'В общем, вся газета.',
 'Православие... более всего подходит на роль такой идеи...',
 'Нефть за $27/барр. не снится.',
 '']

### Бенчмарки
Много вариантов... Нужно измерять

In [10]:
! pip install lxml

In [11]:
# WARNING: RAM bound task, XML parsing is expensive
# Similar to https://github.com/deepmipt/ru_sentence_tokenizer/blob/master/metrics/calculate.ipynb
import re
from lxml import etree

# \W -> Any non-word character
RE_ENDS_WITH_PUNCT = re.compile(r".*\W$")

OPENCORPORA_FILE = "annot.opcorpora.xml"
sentences = list(etree.parse(OPENCORPORA_FILE).xpath('//source/text()'))
singles = []
compounds = []
s2 = sentences.pop().strip()
singles.append(s2)
while sentences:
    s1 = sentences.pop().strip()
    singles.append(s1)
    if RE_ENDS_WITH_PUNCT.match(s1) and not s1.endswith(':') and not s2.startswith('—'):
        compounds.append((s1, s2))
    s2 = s1
        
print(f'Read {len(singles)} sentences from {OPENCORPORA_FILE}')
        
del sentences

Read 110304 sentences from annot.opcorpora.xml


In [12]:
def check_sent_tokenizer(tokenizer, singles, compounds):
    correct_count_in_singles = 0
    for sentence in singles:
        correct_count_in_singles += len(tokenizer(sentence)) == 1

    correct_count_in_compounds = 0
    for s1, s2 in compounds:
        correct_count_in_compounds += tokenizer(s1 + ' ' + s2) == [s1, s2]

    return (correct_count_in_singles / len(singles), correct_count_in_compounds / len(compounds))

In [13]:
from nltk.tokenize import sent_tokenize
%time singles_score, compounds_score = check_sent_tokenizer(sent_tokenize, singles, compounds)
print(f'sent_tokenizer scores: {singles_score*100:.2f}%, {compounds_score*100:.2f}%')

CPU times: user 10 s, sys: 7.07 ms, total: 10 s
Wall time: 10 s
sent_tokenizer scores: 94.30%, 86.07%


In [14]:
russian_sent_tokenize = lambda s : sent_tokenize(s, language="russian")
%time singles_score, compounds_score = check_sent_tokenizer(russian_sent_tokenize, singles, compounds)
print(f'russian sent_tokenizer scores: {singles_score*100:.2f}%, {compounds_score*100:.2f}%')

CPU times: user 10.3 s, sys: 37 ms, total: 10.3 s
Wall time: 10.3 s
russian sent_tokenizer scores: 96.78%, 88.85%


In [15]:
from razdel import sentenize
razdel_sent_tokenize = lambda text : [s.text for s in sentenize(text)]
%time singles_score, compounds_score = check_sent_tokenizer(razdel_sent_tokenize, singles, compounds)
print(f'razdel scores: {singles_score*100:.2f}%, {compounds_score*100:.2f}%')

CPU times: user 8.07 s, sys: 0 ns, total: 8.07 s
Wall time: 8.07 s
razdel scores: 99.07%, 95.39%


In [16]:
from rusenttokenize import ru_sent_tokenize
deepmipt_sent_tokenize = ru_sent_tokenize
%time singles_score, compounds_score = check_sent_tokenizer(deepmipt_sent_tokenize, singles, compounds)
print(f'deepmipt scores: {singles_score*100:.2f}%, {compounds_score*100:.2f}%')

CPU times: user 9.45 s, sys: 0 ns, total: 9.45 s
Wall time: 9.45 s
deepmipt scores: 98.73%, 93.42%


Аналогичные бенчмарки:
- https://github.com/natasha/razdel/blob/master/eval.ipynb
- https://github.com/deepmipt/ru_sentence_tokenizer/blob/master/metrics/calculate.ipynb

### Задание 1: "Кирпич"
Скачайте предложенный текст. Найдите первое предложение, которое отличается в разбиениях, порождённых rusenttokenize и razdel. Верните номер этого предложения.

In [17]:
!wget https://www.dropbox.com/s/q5wo34gfbepc7am/htbg.txt

--2023-03-08 16:33:49--  https://www.dropbox.com/s/q5wo34gfbepc7am/htbg.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.70.18, 2620:100:6026:18::a27d:4612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.70.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/q5wo34gfbepc7am/htbg.txt [following]
--2023-03-08 16:33:50--  https://www.dropbox.com/s/raw/q5wo34gfbepc7am/htbg.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc3ba613fd7318c690d1959535cc.dl.dropboxusercontent.com/cd/0/inline/B30BVqfuJT8uqnzcQlPBCmaoYhQQX2iw6guPW4XT91lG69rnPjco9G1W_pGvHDhHtRpm3qQC8V57suUyxLrqPPqvXMm0rTJbNWZl4lS7F7zq2HrWZ795YCi3j6Z62dv3OZswg6MnE7R34xRqU05P0bsn9J3w1EBaE5jKBErSUtbQjQ/file# [following]
--2023-03-08 16:33:50--  https://uc3ba613fd7318c690d1959535cc.dl.dropboxusercontent.com/cd/0/inline/B30BVqfuJT8uqnzcQlPBCmaoYhQQX2iw6guPW4XT91lG69rnPjco9G1W_pGvHDhHtRpm3qQC8V57suUyxLrqPPqvX

In [18]:
from razdel import sentenize
from rusenttokenize import ru_sent_tokenize

with open("htbg.txt", "r") as f:
    text = f.read()

In [19]:
def get_first_different_sentence(text: str) -> int:
    split_razdel = [sent.text for sent in list(sentenize(text))]
    split_rusent = ru_sent_tokenize(text)
    idx = 0
    for a, b in zip(split_razdel, split_rusent):
        if a == b: 
            idx += 1
            continue
        else: return idx
    return -1

assert get_first_different_sentence(text) == 329

In [20]:
get_first_different_sentence(text)

329

### Задание 2: Lazy baseline
Напишите свой sent_tokenize, который будет делить предложения только по точкам, восклицательным и вопросительным знакам. Измерьте для него время работы и метрики на opencorpora.

In [21]:
def my_sent_tokenize(text):
    sents = []
    init_ch = 0
    i_add = 1
    for i, char in enumerate(text):
        if char in ['.', '?', '!', '...', '!..', '?..']:
            if i < len(text) - 1 and text[i + 1] in ['?', '!', '.']:
                i_add += 1
            sent = text[init_ch:i+i_add]
            sents.append(sent.strip())
            init_ch = i + i_add
    return sents

In [22]:
example1

"this's a sent tokenize test. this is sent two. is this sent three? sent 4 is cool! Now it’s your turn."

In [23]:
my_sent_tokenize(example1)

["this's a sent tokenize test.",
 'this is sent two.',
 'is this sent three?',
 'sent 4 is cool!',
 'Now it’s your turn.']

In [24]:
sent_tokenize(example1)

["this's a sent tokenize test.",
 'this is sent two.',
 'is this sent three?',
 'sent 4 is cool!',
 'Now it’s your turn.']

In [25]:
from nltk.tokenize import sent_tokenize


assert my_sent_tokenize(example1) == sent_tokenize(example1)

%time singles_score, compounds_score = check_sent_tokenizer(my_sent_tokenize, singles, compounds)

# assert singles_score >= 0.85

print(f'your scores: {singles_score*100:.2f}%, {compounds_score*100:.2f}%')

CPU times: user 5.61 s, sys: 3.38 ms, total: 5.62 s
Wall time: 5.62 s
your scores: 80.16%, 72.19%


# Токенизация

Самый наивный способ токенизировать текст -- разделить с помощью split. Но split упускает очень много всего, например, банально не отделяет пунктуацию от слов. Кроме этого, есть ещё много менее тривиальных проблем. Поэтому лучше использовать готовые токенизаторы.

In [26]:
from nltk.tokenize import word_tokenize
print(word_tokenize(example5))

['Good', 'muffins', 'cost', '$', '3.88', 'in', 'New', 'York', '.', 'Please', 'buy', 'me', 'two', 'of', 'them', '.', 'Thanks', '.']


In [27]:
from nltk import tokenize
dir(tokenize)[:16]

['BlanklineTokenizer',
 'LegalitySyllableTokenizer',
 'LineTokenizer',
 'MWETokenizer',
 'NLTKWordTokenizer',
 'PunktSentenceTokenizer',
 'RegexpTokenizer',
 'ReppTokenizer',
 'SExprTokenizer',
 'SpaceTokenizer',
 'StanfordSegmenter',
 'SyllableTokenizer',
 'TabTokenizer',
 'TextTilingTokenizer',
 'ToktokTokenizer',
 'TreebankWordDetokenizer']

Они умеют выдавать индексы начала и конца каждого токена:

In [28]:
from nltk import tokenize
wh_tok = tokenize.WhitespaceTokenizer()
print(list(wh_tok.span_tokenize(example5)))

[(5, 9), (10, 17), (18, 22), (23, 28), (29, 31), (32, 35), (36, 41), (43, 49), (50, 53), (54, 56), (57, 60), (61, 63), (64, 69), (70, 77)]


Некторые токенизаторы ведут себя специфично:

In [29]:
tokenize.TreebankWordTokenizer().tokenize("don't stop me")

['do', "n't", 'stop', 'me']

In [30]:
import spacy
spacy_nlp = spacy.load('en_core_web_sm')
doc = spacy_nlp(example5, disable=["parser"])
print([token.text for token in doc])

['\n    ', 'Good', 'muffins', 'cost', '$', '3.88', 'in', 'New', 'York', '.', ' ', 'Please', 'buy', 'me', 'two', 'of', 'them', '.', 'Thanks', '.', '\n']


In [31]:
from nltk.tokenize import word_tokenize
print(word_tokenize(example4))

['Кружка-термос', 'на', '0.5л', '(', '50/64', 'см³', ',', '516', ';', '...', ')', 'стоит', '$', '3.88']


In [32]:
from razdel import tokenize
list(tokenize(example4))

[Substring(5, 18, 'Кружка-термос'),
 Substring(19, 21, 'на'),
 Substring(22, 25, '0.5'),
 Substring(25, 26, 'л'),
 Substring(27, 28, '('),
 Substring(28, 33, '50/64'),
 Substring(34, 37, 'см³'),
 Substring(37, 38, ','),
 Substring(39, 42, '516'),
 Substring(42, 43, ';'),
 Substring(43, 46, '...'),
 Substring(46, 47, ')'),
 Substring(48, 53, 'стоит'),
 Substring(54, 55, '$'),
 Substring(55, 59, '3.88')]

### Задание 3: Diff
Напишите функцию, которая будет выводить разницу между токенизацией razdel'а и nltk.

In [33]:
from difflib import SequenceMatcher, Differ # USE THIS
from razdel import tokenize
from nltk.tokenize import word_tokenize

with open("htbg.txt", "r") as f:
    text = f.read()

In [34]:
split_razdel = [row.text for row in list(tokenize(text))]
split_razdel[:5]

['Аркадий', 'и', 'Борис', 'Стругацкие', 'Трудно']

In [35]:
split_nltk = word_tokenize(text)
split_nltk[:5]

['Аркадий', 'и', 'Борис', 'Стругацкие', 'Трудно']

In [36]:
len(split_razdel)

64278

In [37]:
len(split_nltk)

63418

In [38]:
seq = SequenceMatcher(a=split_razdel, b=split_nltk)
seq.ratio()

0.9818788372384413

In [39]:
len(seq.get_matching_blocks())

615

In [40]:
list(Differ().compare(split_razdel[0], split_nltk[0]))

['  А', '  р', '  к', '  а', '  д', '  и', '  й']

In [41]:
def get_tokenization_differences(text: str) -> int:
    split_razdel = [row.text for row in list(tokenize(text))]
    split_nltk = word_tokenize(text)
    seq = SequenceMatcher(a=split_razdel, b=split_nltk)
    return seq.get_matching_blocks()
    
print(len(get_tokenization_differences(text)))

615


In [42]:
assert len(get_tokenization_differences(text)) == 615 # 613

# Стоп-слова и пунктуация

Стоп-слова - это слова, которые часто встречаются практически в любом тексте и ничего интересного не говорят о конкретном документе, то есть играют роль шума. Поэтому их принято убирать. По той же причине убирают и пунктуацию.

In [43]:
from nltk.corpus import stopwords
print(stopwords.words('russian'))

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

In [44]:
from string import punctuation
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [45]:
noise = stopwords.words('russian') + list(punctuation)

In [46]:
noise[:10]

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со']

### Задание 4: Стоп-слова from scratch
Постройте свой список стоп-слов на основе Opencorpora

In [47]:
import re
from lxml import etree
from razdel import tokenize

OPENCORPORA_FILE = "annot.opcorpora.xml"
sentences = list(etree.parse(OPENCORPORA_FILE).xpath('//source/text()'))
print(f'Read {len(sentences)} sentences from {OPENCORPORA_FILE}')

Read 110304 sentences from annot.opcorpora.xml


In [48]:
sent = [row.text for row in list(tokenize(text.lower()))]
sent[:5]

['аркадий', 'и', 'борис', 'стругацкие', 'трудно']

In [49]:
from collections import Counter
sent_counts = Counter(sent)

In [50]:
stop_words = [w[0] for w in sent_counts.most_common(n=100)]

In [51]:
stop_words[:5]

[',', '.', '–', 'и', 'в']

In [52]:
del sentences

# Стемминг

In [53]:
from nltk.stem.snowball import SnowballStemmer 
from razdel import tokenize

stemmer = SnowballStemmer("russian") 
print([stemmer.stem(token.text) for token in tokenize(example3)])

['а', 'что', 'насчет', 'русск', 'язык', '?', 'хорош', 'ли', 'сегментир', 'им', '?', 'а', 'да', 'а', '.', 'с', '.', 'пушкин', '!', 'а', 'да', 'сукин', 'сын', '!', '«', 'как', 'же', 'так', '?!', 'захар', '...', '»', '—', 'воскликнут', 'пронин', '.', '-', '"', 'так', 'в', 'чем', 'же', 'дел', '?', '"', '-', '"', 'не', 'ра-ду-ют', '"', '.', 'и', 'т', '.', 'д', '.', 'и', 'т', '.', 'п', '.', 'в', 'общ', ',', 'вся', 'газет', '.', 'православ', '...', 'бол', 'всег', 'подход', 'на', 'рол', 'так', 'ид', '...', 'нефт', 'за', '$', '27', '/', 'барр', '.', 'не', 'снит', '.']


# Лемматизация и морфологический анализ

Лемматизация – это сведение разных форм одного слова к начальной форме – лемме. Почему это хорошо?
* Мы хотим рассматривать как отдельную фичу каждое слово, а не каждую его отдельную форму.
* Некоторые стоп-слова стоят только в начальной форме, и без лемматизации выкидываем мы только её.

Для русского есть два хороших лемматизатора: mystem и pymorphy. С pymorphy всё сразу понятно.

Но как работать с Mystem:
* Можно скачать mystem и запускать из терминала с разными параметрами
* pymystem3 - обертка для питона, работает медленнее, но это удобно

## Mystem

In [54]:
# ! pip install pymystem3

In [56]:
from pymystem3 import Mystem
mystem_analyzer = Mystem()

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [57]:
! chmod +x /root/.local/bin/mystem

Мы инициализировали Mystem c дефолтными параметрами. А вообще параметры есть такие:

    mystem_bin - путь к mystem, если их несколько
    grammar_info - нужна ли грамматическая информация или только леммы (по дефолту нужна)
    disambiguation - нужно ли снятие омонимии - дизамбигуация (по дефолту нужна)
    entire_input - нужно ли сохранять в выводе все (пробелы всякие, например), или можно выкинуть (по дефолту оставляется все)

Методы Mystem принимают строку, токенизатор вшит внутри. Можно, конечно, и пословно анализировать, но тогда он не сможет учитывать контекст.

Можно просто лемматизировать текст:

In [58]:
print(mystem_analyzer.lemmatize(example3))

['\n', '    ', 'а', ' ', 'что', ' ', 'насчет', ' ', 'русский', ' ', 'язык', '? ', 'хорошо', ' ', 'ли', ' ', 'сегментироваться', ' ', 'имя', '?', '\n', '    ', 'ай', ' ', 'да', ' ', 'а', '.', 'с', '. ', 'пушкин', '! ', 'ай', ' ', 'да', ' ', 'сукин', ' ', 'сын', '!', '\n', '    «', 'как', ' ', 'же', ' ', 'так', '?! ', 'захар', '...', '» — ', 'восклицать', ' ', 'пронин', '.', '\n', '    - "', 'так', ' ', 'в', ' ', 'чем', ' ', 'же', ' ', 'дело', '?" - ', '"', 'не', ' ', 'ра', '-', 'ду', '-', 'ют', '"', '.', '\n', '    ', 'и', ' ', 'т', '.', ' ', 'д', '.', ' ', 'и', ' ', 'т', '.', ' ', 'п', '. ', 'в', ' ', 'общий', ', ', 'весь', ' ', 'газета', '.', '\n', '    ', 'православие', '...', ' ', 'много', ' ', 'все', ' ', 'подходить', ' ', 'на', ' ', 'роль', ' ', 'такой', ' ', 'идея', '...', '\n', '    ', 'нефть', ' ', 'за', ' ', '$', '27', '/', 'барра', '.', ' ', 'не', ' ', 'сниться', '.', '\n']


А можно получить грамматическую информацию:

In [59]:
mystem_analyzer.analyze(example3)

[{'text': '\n'},
 {'text': '    '},
 {'analysis': [{'lex': 'а', 'wt': 0.9822148501, 'gr': 'CONJ='}], 'text': 'А'},
 {'text': ' '},
 {'analysis': [{'lex': 'что',
    'wt': 0.2934446278,
    'gr': 'SPRO,ед,сред,неод=(вин|им)'}],
  'text': 'что'},
 {'text': ' '},
 {'analysis': [{'lex': 'насчет', 'wt': 1, 'gr': 'PR='}], 'text': 'насчёт'},
 {'text': ' '},
 {'analysis': [{'lex': 'русский',
    'wt': 0.9496245492,
    'gr': 'A=(вин,ед,полн,муж,од|род,ед,полн,муж|род,ед,полн,сред)'}],
  'text': 'русского'},
 {'text': ' '},
 {'analysis': [{'lex': 'язык', 'wt': 0.9963354032, 'gr': 'S,муж,неод=род,ед'}],
  'text': 'языка'},
 {'text': '? '},
 {'analysis': [{'lex': 'хорошо', 'wt': 0.0008292704217, 'gr': 'ADV=вводн'}],
  'text': 'Хорошо'},
 {'text': ' '},
 {'analysis': [{'lex': 'ли', 'wt': 0.7719288688, 'gr': 'PART='}],
  'text': 'ли'},
 {'text': ' '},
 {'analysis': [{'lex': 'сегментироваться',
    'wt': 1,
    'gr': 'V,несов,нп=непрош,мн,изъяв,3-л'}],
  'text': 'сегментируются'},
 {'text': ' '},
 {

## Pymorphy

Это модуль на питоне, довольно быстрый и с кучей функций.

In [61]:
! pip install pymorphy2

     |████████████████████████████████| 55 kB 1.1 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 8.2 MB 2.6 MB/s            
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=23d1b52562ed6f73b4c890e51e3e44b2cf04e5f7994edd9a5be02be27f78ebc9
  Stored in directory: /root/.cache/pip/wheels/3f/2a/fa/4d7a888e69774d5e6e855d190a8a51b357d77cc05eb1c097c9
Successfully built docopt


In [62]:
from pymorphy2 import MorphAnalyzer
pymorphy2_analyzer = MorphAnalyzer()

In [63]:
pymorphy2_analyzer.parse("мою")

[Parse(word='мою', tag=OpencorporaTag('ADJF,Apro femn,sing,accs'), normal_form='мой', score=0.970588, methods_stack=((DictionaryAnalyzer(), 'мою', 2049, 10),)),
 Parse(word='мою', tag=OpencorporaTag('VERB,impf,tran sing,1per,pres,indc'), normal_form='мыть', score=0.029411, methods_stack=((DictionaryAnalyzer(), 'мою', 2074, 1),))]

In [69]:
res = pymorphy2_analyzer.parse("мою")

In [74]:
res[0].tag.POS

'ADJF'

In [75]:
res[0].word

'мою'

### Задание 5: Анализ частей речи

Используя pymorphy2, определите топ-10 самых частотных существительных и глаголов в тексте

In [64]:
from pymorphy2 import MorphAnalyzer

pymorphy2_analyzer = MorphAnalyzer()
with open("htbg.txt", "r") as f:
    text = f.read()
# YOUR CODE HERE

In [67]:
sent = [row.text for row in list(tokenize(text.lower()))]
sent[:5]

['аркадий', 'и', 'борис', 'стругацкие', 'трудно']

In [77]:
from tqdm.notebook import tqdm

In [79]:
adjectives = []
for word in tqdm(sent):
    res = pymorphy2_analyzer.parse(word)
    for i, w in enumerate(res):
        if res[i].tag.POS == 'ADJF':
            adjectives.append(res[i].word)

  0%|          | 0/64278 [00:00<?, ?it/s]

In [80]:
adjectives[:5]

['то', 'то', 'его', 'его', 'его']

In [81]:
from collections import Counter
adjectives_counts = Counter(adjectives)

In [82]:
adjectives_most_common = [w[0] for w in adjectives_counts.most_common(n=100)]

In [84]:
adjectives_most_common[:20]

['его',
 'рэба',
 'их',
 'её',
 'это',
 'все',
 'всё',
 'цупик',
 'нем',
 'то',
 'благородный',
 'этого',
 'мой',
 'всех',
 'другой',
 'один',
 'такой',
 'всего',
 'этой',
 'сам']

## mystem vs. pymorphy

1) Mystem работает невероятно медленно под windows на больших текстах.

2) Снятие омонимии. Mystem умеет снимать омонимию по контексту (хотя не всегда преуспевает), pymorphy2 берет на вход одно слово и соответственно вообще не умеет дизамбигуировать по контексту:


In [85]:
homonym1 = 'За время обучения я прослушал больше сорока курсов.'
homonym2 = 'Сорока своровала блестящее украшение со стола.'
mystem_analyzer = Mystem() # инициализирую объект с дефолтными параметрами

print(mystem_analyzer.analyze(homonym1)[-5])
print(mystem_analyzer.analyze(homonym2)[0])

{'analysis': [{'lex': 'сорок', 'wt': 0.8710292664, 'gr': 'NUM=(пр|дат|род|твор)'}], 'text': 'сорока'}
{'analysis': [{'lex': 'сорока', 'wt': 0.1210970041, 'gr': 'S,жен,од=им,ед'}], 'text': 'Сорока'}


## Rnnmorph
Обёртка над pymorphy с разрешением омонимии

https://github.com/IlyaGusev/rnnmorph

https://habr.com/ru/post/339954/

In [87]:
! pip install rnnmorph

     |████████████████████████████████| 19.7 MB 146 kB/s              
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for rnnmorph: filename=rnnmorph-0.4.1-py3-none-any.whl size=19746379 sha256=d3927e686850c661460f813c539d4a598abe707bceb019b278dd432d11f48e29
  Stored in directory: /root/.cache/pip/wheels/34/cc/02/6a434b98af4e1129902b679a3163b5ca2e56b16244378fdc8c
  Created wheel for russian-tagsets: filename=russian_tagsets-0.6-py3-none-any.whl size=24636 sha256=57c67e37b33f1bf02605db24647dc2d53d55e50968dacf091e415e30ae2440c9
  Stored in directory: /root/.cache/pip/wheels/2e/a7/fe/2aca8a961e93604384b1e0c94925023c9d17539873b433e8fb
Successfully built rnnmorph russian-tagsets


In [88]:
from rnnmorph.predictor import RNNMorphPredictor
from razdel import tokenize

predictor = RNNMorphPredictor(language="ru")
homonym = "Косил косой косой косой"
print(predictor.predict([t.text for t in tokenize(homonym)])[1])
print(predictor.predict([t.text for t in tokenize(homonym)])[-1])

AttributeError: module 'tensorflow.compat.v2.__internal__' has no attribute 'register_clear_session_function'

## GramEval-2020

Соревнование по определению морфологических характеристик, определению синтаксических зависимостей и лемматизации. Готовых инструментов не получилось, но весь код всех конкурсантов доступен.
* https://github.com/dialogue-evaluation/GramEval2020

### Задание 6: Формат

Используя стандартные инструменты переведите корпус htbg.txt в формат CoNLL-U.
Используйте следующие колонки: 
    1. Номер предложения в тексте
    2. Токен в том виде, в котором он встретился в тексте
    3. Лемма токена
    4. POS-таг токена
    5. Вектор грамматических значений токена
    6. Целевая метка (сделайте метку везде OUT)

# Regex 101

In [89]:
import re

#### match
ищет по заданному шаблону в начале строки

In [90]:
result = re.match('ab+c.', 'abcdefghijkabcabc') # ищем по шаблону 'ab+c.' 
print (result) # совпадение найдено:

<_sre.SRE_Match object; span=(0, 4), match='abcd'>


In [91]:
print(result.group(0)) # выводим найденное совпадение

abcd


In [92]:
result = re.match('abc.', 'abdefghijkabcabc')
print(result) # совпадение не найдено

None


#### search
ищет по всей строке, возвращает только первое найденное совпадение

In [93]:
result = re.search('ab+c.', 'aefgabchijkabcabc') 
print(result) 

<_sre.SRE_Match object; span=(4, 8), match='abch'>


#### findall
возвращает список всех найденных совпадений

In [94]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc') 
print(result)

['abcd', 'abca']


Вопросы: 
1) почему нет последнего abc?
2) почему нет abcx?

#### split
разделяет строку по заданному шаблону


In [95]:
result = re.split(',', 'itsy, bitsy, teenie, weenie') 
print(result)

['itsy', ' bitsy', ' teenie', ' weenie']


можно указать максимальное количество разбиений

In [96]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit = 2) 
print(result)

['itsy', ' bitsy', ' teenie, weenie']


#### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [97]:
result = re.sub('a', 'b', 'abcabc')
print (result)

bbcbbc


#### compile
компилирует регулярное выражение в отдельный объект

In [98]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'Да', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

In [ ]:
# Ваш код